In [16]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import time



In [17]:
# Put on GPU

device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

print(noisy_imgs_1.size())


torch.Size([50000, 3, 32, 32])


In [19]:
# selct a preset of images:

imgs_1 = noisy_imgs_1[:10000]/255
imgs_2 = noisy_imgs_2[:10000]/255


In [27]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        """
        self.conv1 = nn.Conv2d(3, 10, kernel_size = 5, padding = (5 - 1) // 2)
        self.conv2 = nn.Conv2d(10, 10, kernel_size = 5, padding = (5 - 1) // 2)
        self.convT1 = nn.ConvTranspose2d(10, 3, kernel_size = 5, padding = (5 - 1) // 2)
        """
        self.conv1 = nn.Conv2d(3, 10, kernel_size = 5, stride = 1)
        self.conv2 = nn.Conv2d(10, 10, kernel_size = 4, stride = 2)
        self.convT1 = nn.ConvTranspose2d(10, 10, kernel_size = 4, stride = 2)
        self.convT2 = nn.ConvTranspose2d(10, 3, kernel_size = 5, stride = 1)
        
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        #print('x',x.shape)
        x = self.dropout(F.relu(self.conv2(x)))
        #print('x',x.shape)
        x = self.dropout(F.relu(self.convT1(x)))
        #print('x',x.shape)
        x = F.sigmoid(self.convT2(x))

        return x

In [28]:
### For mini - project 1
class Model(nn.Module):
    
    def __init__(self):
    ## instantiate model + optimizer + loss function + any other stuff you need
        super().__init__()
        
        self.device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
        self.autoenc = Net().to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.autoenc.parameters(), lr = 1e-2)
        
        pass

    def load_pretrained_model(self):
    ## This loads the parameters saved in bestmodel .pth into the model
        self.autoenc  = model.load_state_dict(torch.load('bestmodel.pth'),strict=False)
        pass

    def train(self , train_input , train_target , mini_batch_size):
    #: train˙input : tensor of size (N, C, H, W) containing a noisy version of the images.

        train_input, train_target = train_input.to(self.device), train_target.to(self.device) # Use GPU
        
        nb_epochs = 20
        #et

        for epoch in range(nb_epochs):
            acc_loss = 0

            for b in range(0, train_input.size(0), mini_batch_size):
                output = self.autoenc(train_input.narrow(0, b, mini_batch_size))
                loss = self.criterion(output, train_target.narrow(0, b, mini_batch_size))
                acc_loss = acc_loss + loss.item()
                #print('output', output.shape)
                #print('loss',loss.shape, loss)
                #print('acc loss', acc_loss)

                #self.autoenc.zero_grad()
                self.optimizer.zero_grad()
                loss.backward()
                #print('optimiser',self.optimizer)
                #print(loss.grad)
                #print(self.autoenc.conv1.input.grad.shape)
                print('grad',self.autoenc.conv2.weight.grad[0,0][0])
                #print(self.autoenc.convT1.weight.grad.shape)
                #print(self.autoenc.convT2.weight.grad.shape)
                
                self.optimizer.step()
                """
                with torch.no_grad():
                    for p in self.autoenc.parameters():
                        p -= eta * p.grad
                """
                        
            if not epoch % 5: 
                print(epoch, acc_loss)

    #: train˙target : tensor of size (N, C, H, W) containing another noisy version of the same images , which only differs from the input by their noise .
        pass

    def predict(self , test_input ):
    #: test˙input : tensor of size (N1 , C, H, W) that has to be denoised by the trained or the loaded network .
    #: returns a tensor of the size (N1 , C, H, W)
    
        torch.save(self.autoenc.state_dict(), 'bestmodel.pth')
        output = self.autoenc(test_input)
        
        return output
    
    # Add a method "load pretrained model" to load the best model


In [29]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)

In [30]:
mini_batch_size = 1000

model = Model()
print('ici')
model.train( imgs_1, imgs_2, mini_batch_size)

out = model.predict(noisy_imgs)

img_denoise_psnr = psnr(out ,clean_imgs )


print("psnr", img_denoise_psnr)


ici
grad tensor([-5.0753e-05, -5.5815e-05, -6.2262e-05, -6.2338e-05])
grad tensor([0.0009, 0.0010, 0.0010, 0.0010])
grad tensor([-1.2351e-04, -1.0401e-04, -9.3690e-05, -8.0955e-05])
grad tensor([-0.0002, -0.0002, -0.0002, -0.0002])
grad tensor([-0.0011, -0.0011, -0.0011, -0.0010])
grad tensor([0.0016, 0.0016, 0.0016, 0.0017])
grad tensor([-0.0044, -0.0044, -0.0044, -0.0043])
grad tensor([-0.0030, -0.0031, -0.0031, -0.0030])
grad tensor([0.0076, 0.0075, 0.0076, 0.0078])
grad tensor([-0.0006, -0.0007, -0.0007, -0.0005])
0 0.7078900299966335
grad tensor([-0.0104, -0.0105, -0.0105, -0.0103])
grad tensor([-0.0017, -0.0019, -0.0018, -0.0017])
grad tensor([0.0122, 0.0121, 0.0121, 0.0124])
grad tensor([0.0047, 0.0046, 0.0047, 0.0048])
grad tensor([-0.0064, -0.0065, -0.0065, -0.0064])
grad tensor([-0.0115, -0.0116, -0.0116, -0.0114])
grad tensor([-0.0085, -0.0087, -0.0086, -0.0085])
grad tensor([0.0009, 0.0008, 0.0008, 0.0010])
grad tensor([0.0102, 0.0100, 0.0100, 0.0102])
grad tensor([0.0069, 

grad tensor([-9.6622e-05, -9.3630e-05, -1.1984e-04, -1.3259e-04])
grad tensor([0.0008, 0.0008, 0.0008, 0.0008])
grad tensor([-0.0009, -0.0009, -0.0009, -0.0009])
grad tensor([0.0010, 0.0011, 0.0010, 0.0010])
grad tensor([-0.0006, -0.0006, -0.0006, -0.0006])
grad tensor([ 6.9177e-05,  6.0611e-05,  2.7850e-05, -9.8424e-07])
grad tensor([0.0005, 0.0005, 0.0005, 0.0005])
grad tensor([-0.0007, -0.0007, -0.0007, -0.0008])
grad tensor([0.0008, 0.0009, 0.0008, 0.0008])
grad tensor([-0.0006, -0.0006, -0.0006, -0.0006])
grad tensor([0.0005, 0.0005, 0.0005, 0.0005])
grad tensor([1.3804e-04, 1.3105e-04, 1.0369e-04, 7.4398e-05])
grad tensor([-0.0003, -0.0002, -0.0003, -0.0003])
grad tensor([0.0005, 0.0005, 0.0005, 0.0005])
grad tensor([-0.0002, -0.0003, -0.0003, -0.0003])
grad tensor([ 5.0222e-05,  5.3899e-05,  1.9681e-05, -8.7227e-06])
grad tensor([7.1691e-05, 7.4042e-05, 5.8588e-05, 3.7672e-05])
grad tensor([7.5184e-05, 8.7086e-05, 6.2170e-05, 3.8020e-05])
grad tensor([ 7.1482e-06,  6.2985e-06,  

In [ ]:
# AFFICHAGE
print(out[1].shape)
"""
#print(clean_imgs[1])
plt.imshow(clean_imgs[1].permute(1,2,0))
plt.show()
plt.imshow(out[1].permute(1,2,0).detach())
plt.show()
"""
fig, axs = plt.subplots(1,3,figsize=(15,15))

axs[0].imshow(noisy_imgs[1].permute(1,2,0))
axs[0].set_title('noisy')
axs[1].imshow(clean_imgs[1].permute(1,2,0))
axs[0].set_title('clean')
axs[2].imshow(out[1].permute(1,2,0).detach())
axs[0].set_title('predict')



In [67]:
model_2 = Model()
model_2.load_pretrained_model()
out = model_2.predict(noisy_imgs)

TypeError: cannot assign 'torch.nn.modules.module._IncompatibleKeys' as child module 'autoenc' (torch.nn.Module or None expected)